# Process ASSISTment 2009-2010 Dataset

In [1]:
import pandas as pd 
import numpy as np
from collections import defaultdict
from utils import min_seq_len_filter

In [2]:
MIN_SEQ_LENGTH = 10

In [ ]:
df = pd.read_csv("./raw-datasets/skill_builder_data_corrected_collapsed.csv", engine='python')

In [ ]:
df = df[~df['skill_name'].isnull() & (df['answer_type'] != 'open_response')]
df.sort_values('order_id', inplace=True)    

In [ ]:
def make_indecies(series):
    unique_vals = sorted(set(series))
    return dict(zip(unique_vals, range(len(unique_vals))))
def make_mapping(keys_series, vals_series):
    mapping = defaultdict(set)
    for k, v in zip(keys_series, vals_series):
        mapping[k].add(v)
    return dict(mapping)

In [ ]:
problem_id = make_indecies(df['problem_id'])
student_id = make_indecies(df['user_id'])
skill_id = make_indecies(df['skill_id'])
answer_type_id = make_indecies(df['answer_type'])
sid_to_sname = make_mapping(df['skill_id'], df['skill_name'].str.strip())

In [ ]:
rows = []
for r in df.itertuples():
    rows.append({
        "student" : student_id[r.user_id],
        "skill" : skill_id[r.skill_id],
        "correct" : r.correct,
        "problem" : problem_id[r.problem_id],
        "answer_type" : answer_type_id[r.answer_type],
        "response_time_ms" : r.ms_first_response,
        "skill_name" : r.skill_name
    })
output_df = pd.DataFrame(rows)
output_df = min_seq_len_filter(output_df, MIN_SEQ_LENGTH)

In [ ]:
print("Trials: %d" % output_df.shape[0])
print("Students: %d" % len(set(output_df['student'])))
print("Skills: %d" % len(set(output_df['skill'])))
print("Problems: %d" % len(set(output_df['problem'])))
df.to_csv("datasets/assistment.csv", index=False)